In [1]:
import pandas as pd
import numpy as np
import random
import pickle
import sys
from keras.models import load_model
from keras import backend
import json 

sys.path.append("../../src/models/predict_model")
import predictor
sys.path.append("../../src/data")
import exclusionCriteria

pd.options.mode.chained_assignment = None  # default='warn'


Using TensorFlow backend.


## Get one record for a sample prediction

In [2]:
fileName = 'ED_TOTAL_2009_2009.csv'
with open('../../fileConfig.json') as config_file:    
        fileConfig = json.load(config_file)
fileLocation = fileConfig['dataDirectory'] + fileConfig['processedDirectory'] + fileName
cdc_file = pd.read_csv(fileLocation)        

In [3]:
# records with vital signs = -9 are filtered out for demo purposes
# todo: resolve records with missing data 
reload(exclusionCriteria)
cdc_file = exclusionCriteria.filter_out_records_na(cdc_file)


In [4]:
#getting one record from the input file, as an example
record_index = random.randint(0,len(cdc_file))
sample_record = cdc_file.iloc[[record_index]]
sample_record.head()

,VYEAR,VMONTH,VDAYR,AGE,ARRTIME,WAITTIME,LOV,RESIDNCE,SEX,ETHUN,...,ADMITHOS,OBSHOS,OBSDIS,OTHDISP,ADMIT,HDSTAT,BDATEFL,IMMEDRFL,REGION,MSA
3229,2009,4,5,33,1952,102,124,1,2,1,...,0,0,0,0,-7,-7,0,0,1,1


##  Read models 

In [5]:
with open('../../modelConfig.json') as config_file:    
        modelConfig = json.load(config_file)
        
modelConfig

{u'co_model': u'../../models/cdc_2009_nn_att_text_embedding.H5',
 u'max_text_length': u'../../models/cdc_2009_att_text_max_length.pickle',
 u'rss_model': u'../../models/cdc_rss_2009_nn_att_text_embedding.H5',
 u'tokenizer': u'../../models/cdc_2009_att_text_tokenizer.pickle'}

## Thresholds

In [6]:
fileESIThreshold_location = fileConfig['dataDirectory'] + fileConfig['resultDirectory'] + 'esi_thresholds.json'  
with open(fileESIThreshold_location) as config_file:    
        thresholdsConfig = json.load(config_file)
thresholdsConfig

{u'esi1_threshold': 0.8709694147109985,
 u'esi2_threshold': 0.7043555974960327,
 u'rss_threshold': 0.35}

## Pass data to predictor

In [7]:
json_data = sample_record.to_json(orient='records')

In [8]:
reload(predictor)
att_top = 15
output = predictor.predict(modelConfig,thresholdsConfig, json_data, att_top)


/home/zenobiosky/anaconda2/lib/python2.7/site-packages/keras/engine/topology.py:1269: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


Creating text for embeddings


In [9]:
print(output)

{"co": 0.04415686056017876, "rss": [0.0], "esi": [5], "co_fi": [{"input_value": "2: NOT by ambulance", "featureName": "arrival_model", "attention_weight": 0.021739035844802856}, {"input_value": "1", "featureName": "Reason_Musculoskeletal_Other", "attention_weight": 0.021616321057081223}, {"input_value": "1", "featureName": "Age_31_40_range", "attention_weight": 0.02074449695646763}, {"input_value": "1", "featureName": "Moderate_Tachypnea", "attention_weight": 0.01969575323164463}, {"input_value": "0", "featureName": "chf_indicator", "attention_weight": 0.017187701538205147}, {"input_value": "0", "featureName": "Age_81_Above", "attention_weight": 0.0150623032823205}, {"input_value": null, "featureName": "reason2_for_visit", "attention_weight": 0.0150111960247159}, {"input_value": "0", "featureName": "Reason_Shortness_of_Breath", "attention_weight": 0.014397731050848961}, {"input_value": "0", "featureName": "Age_18_30_range", "attention_weight": 0.012177063152194023}, {"input_value": "0"